# imports

In [2]:
import requests
import pandas as pd

# acquire data

In [47]:
api_url = "https://opendata.maryland.gov/resource/crti-ybyp.json?$limit=5000"

In [48]:
resp = requests.get(api_url)
data = resp.json()

In [49]:
data[0]

{'sno': '1',
 'complaint': '232264',
 'complaint_description': 'concern of loud noise in the Cheverly area',
 'complaint_type': 'Other',
 'recieved_date': '2024-03-06',
 'incident_date': '2024-03-06',
 'county': "Prince George's",
 'incident_closed_date': '2024-03-08',
 'incident_status_desc': 'Incident Closed-Managed'}

In [50]:
df = pd.DataFrame(data)

# clean data

In [51]:
backup_df = df.copy()

In [52]:
df['recieved_date'] = pd.to_datetime(df['recieved_date'])
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['incident_closed_date'] = pd.to_datetime(df['incident_closed_date'])

In [53]:
df.dtypes

sno                              object
complaint                        object
complaint_description            object
complaint_type                   object
recieved_date            datetime64[ns]
incident_date            datetime64[ns]
county                           object
incident_closed_date     datetime64[ns]
incident_status_desc             object
incident_zip                     object
dtype: object

# analyze data

In [54]:
year_ago_date = pd.Timestamp('2023-03-09')

In [55]:
last_year_df = df[df['recieved_date'] > year_ago_date].copy() # save as

In [65]:
last_year_df['county'].value_counts()

county
Baltimore City        101
Anne Arundel           85
Prince George's        83
Frederick              62
Baltimore              57
Montgomery             35
Cecil                  25
Dorchester             17
Harford                17
Howard                 15
Allegany               15
Washington             14
Charles                12
Carroll                 9
Wicomico                8
Not Yet Determined      7
Worcester               7
St. Mary's              6
Garrett                 5
Caroline                3
Queen Anne's            2
Somerset                2
Talbot                  1
Kent                    1
Statewide               1
Name: count, dtype: int64

In [78]:
last_year_df['complaint_type'] = last_year_df['complaint_type'].str.split(', ') # split complaint type by delimiter

In [79]:
complaints_by_type = last_year_df.explode('complaint_type') # fleshes out column to individual rows
last_year_by_type = complaints_by_type['complaint_type'].value_counts()

In [80]:
invalid = ['Not Yet Determined','Statewide']
invalid_county_rows = last_year_df[last_year_df['county'].isin(invalid)]

In [81]:
invalid_county_rows.to_csv('exported_data/invalid_counties.csv')

In [88]:
last_year_df = last_year_df[last_year_df['county'].isin(invalid) == False]

In [92]:
last_year_by_county = last_year_df['county'].value_counts()

# export data

In [94]:
last_year_by_type.to_csv('exported_data/complaint_type_frequency.csv')
last_year_by_county.to_csv('exported_data/county_frequency.csv')
last_year_df.to_csv('exported_data/all_complaints.csv',index=False)